In [1]:
# Input info
groups_f = ""


input_params = ""

outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/clonal_shifts/clones/"

clone_col = "name"
atac_col = "cluster_labels"
barcodes_dir = ""

# params
min_clone_size = 2
p_thresh = 0.1 
n_shuffle=1000
n_cpus = 8



In [2]:
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from src import clonal_shifts as cs
from icecream import ic

### Setup outdirs

In [ ]:
sim_dir = join(outdir, "shuffle")
if not exists(sim_dir):
    os.mkdir(sim_dir)

## Load groups

In [ ]:
cells_meta = pd.read_csv(groups_f, sep="\t")

In [ ]:
def load_groups(groups_f, clone_col, d, barclodes_dir=None):
    if clone_col == "name":
        cells_meta = pd.read_csv(groups_f, sep="\t", index_col=0)
        if cells_meta.index.duplicated().any():
            cells_meta = cells_meta.reset_index()

        cells_meta = cells_meta.loc[~(cells_meta["name"]=="None")]


    elif clone_col == "den_clust":
        barcodes_in = {}
        barcodes_in[d] = pd.read_csv(join(barcodes_dir,f"donor{d}.clones_dendro.csv"), index_col=0)
        #barcodes_in[d] = str(d) + "_" + barcodes_in[str(d)]
        if clone_col not in barcodes_in[d].columns.values:
            barcodes_in[d][clone_col] = f"{d}_C0"
        else:
            barcodes_in[d][clone_col] = str(d) + "_" + barcodes_in[d][clone_col]
        
        cells_meta = pd.read_csv(groups_f, sep="\t")
        cells_meta = cells_meta.loc[~(cells_meta["name"]=="None")]
        cells_meta[clone_col] = cells_meta.apply(lambda x: barcodes_in[int(x["donor"])].loc[x["name"], clone_col] , axis=1)
        
    if not "cluster_labels" in cells_meta.columns.values:
        cells_meta["cluster_labels"] = cells_meta["seurat_clusters"]

    if condition == "inputOnly":
        cells_meta = cells_meta.loc[cells_meta["condition"]==input_cond]
    else:
        cells_meta = cells_meta.loc[cells_meta["condition"]!=input_cond]

    
    return cells_meta[cells_meta["donor"].astype(str)==str(d)]


## Run hypergeo for each donor

In [ ]:
for d in cells_meta["donor"].unique():
    val = load_groups(groups_f, clone_col, d, barcodes_dir)
    
    curr_groups = val.groupby([atac_col, clone_col]).size().reset_index().rename({0:"count"}, axis=1)
    curr_sizes = val.groupby(clone_col).size().sort_values(ascending=False)
    curr_name_cond_size = val.groupby([clone_col]).size()
    curr_name_cond_size = curr_name_cond_size[curr_name_cond_size>min_clone_size]
    curr_clones_filt = curr_name_cond_size.index 
    curr_sizes = curr_sizes.loc[curr_clones_filt].sort_values(ascending=False)
    
    curr_clones = np.unique(curr_groups[clone_col])
    curr_clone_map = {x:ind for ind,x in enumerate(curr_clones)}
    print(curr_clones)
    curr_atac_cl = np.unique(curr_groups[atac_col])
    print(curr_atac_cl)
    
    curr_don_out = donor_out[int(d)]
    
    # Run init hypergeo and the counts data
    cs.hypergeo_plots(curr_groups, curr_clones, curr_atac_cl, curr_sizes, p_thresh, atac_col,
                   clone_col, curr_don_out)
    
    out_df, hyper_df, results_df, out_d = cs.run_data_and_shuffle(curr_groups, curr_don_out, atac_col, clone_col, p_thresh, curr_clones, 
                                                      curr_atac_cl, n_shuffle=n_shuffle, figs_close=False, to_p_correct=False,
                                                                  n_cpus=n_cpus)
    

In [ ]:
# cs.hypergeo_plots(curr_groups, curr_clones, curr_atac_cl, curr_sizes, p_thresh, atac_col,
#                clone_col, curr_don_out)

# out_df, hyper_df, results_df, out_d = cs.run_data_and_shuffle(curr_groups, curr_don_out, atac_col, clone_col, p_thresh, curr_clones, 
#                                                   curr_atac_cl, n_shuffle=n_shuffle, figs_close=False, to_p_correct=False,
#                                                               n_cpus=n_cpus)


## B) Run using all donors as background

In [ ]:
sizes = cells_meta.groupby(clone_col).size().sort_values(ascending=False)
name_cond_size = cells_meta.groupby([clone_col]).size()
name_cond_size = name_cond_size[name_cond_size>min_clone_size]
clones_filt = name_cond_size.index 

sizes = sizes.loc[clones_filt].sort_values(ascending=False)
clones_filt

groups = cells_meta.groupby([atac_col, clone_col]).size().reset_index().rename({0:"count"}, axis=1)
groups

clones = clones_filt.values #np.unique(groups["name"])
clone_map = {x:ind for ind,x in enumerate(clones)}

atac_cl = np.unique(groups[atac_col])
atac_cl


# Run init hypergeo
cs.hypergeo_plots(groups, clones, atac_cl, sizes, p_thresh, atac_col,
               clone_col, all_dir)
## Run shuffle
print("Running hypergeo shuffle and saving sig results")
out_df, hyper_df, results_df, out_d = cs.run_data_and_shuffle(groups, all_dir, atac_col, clone_col, p_thresh, clones, 
                                                  atac_cl, n_shuffle=n_shuffle, figs_close=False, to_p_correct=False,